In [ ]:
import os, tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src import dataio, visualize, evaluate

In [ ]:
# データセットの読み込み
df_test, images_test = dataio.read_data('../dataset/test')

In [ ]:
visualize.plot_images(images_test[:25], df_test, ncols=5, figsize=(15, 15))

In [ ]:
logistic_data = dataio.read_logistic_weights('../weights/logistic_dict.pkl')

In [ ]:
def preprocessing(images: np.ndarray) -> np.ndarray:
    """画像の前処理を行う関数

    Parameters
    ----------
    images : np.ndarray
        前処理する画像

    Returns
    -------
    np.ndarray
        前処理後の画像
    """
    images = images.astype(np.float32)
    for i in range(images.shape[0]):
        for j in range(images.shape[1]):
            for k in range(images.shape[2]):
                images[i, j, k] = images[i, j, k] / 255.0
    return images

def standardize(images: np.ndarray, mean: np.ndarray, std: np.ndarray) -> np.ndarray:
    """画像の標準化を行う関数

    Parameters
    ----------
    images : np.ndarray
        標準化する画像
    mean : np.ndarray
        平均値
    std : np.ndarray
        標準偏差

    Returns
    -------
    np.ndarray
        標準化後の画像
    """
    images = (images - mean) / std
    return images

In [ ]:
def predict(weights_dict: dict[str, np.ndarray], images: np.ndarray) -> np.ndarray:
    """ロジスティック回帰モデルを用いて画像のクラスを予測する関数

    Parameters
    ----------
    weights_dict : dict[str, np.ndarray]
        ロジスティック回帰モデルの重みと切片
    images : np.ndarray
        予測する画像

    Returns
    -------
    np.ndarray
        予測されたクラス
    """
    weights = weights_dict['weights']
    intercept = weights_dict['intercept']
    
    predictions = []
    for i in range(images.shape[0]):
        maxval, argmax = -np.inf, -1
        for j in range(weights.shape[0]):
            pred = 0
            # 内積
            for k in range(images.shape[1]):
                for l in range(images.shape[2]):
                    pred += images[i, k, l] * weights[j, k*images.shape[2]+l]
            # 切片
            pred += intercept[j]
            if pred > maxval:
                maxval = pred
                argmax = j
        predictions.append(argmax)

    return np.array(predictions)

In [ ]:
y_true = df_test["label"].values

In [ ]:
def run():
    images_test_preprocessed = preprocessing(images_test)
    images_test_preprocessed = standardize(images_test_preprocessed,
                                            mean=logistic_data['mean'], 
                                            std=logistic_data['std'])
    predictions = predict(logistic_data, images_test_preprocessed)
    result = evaluate.calculate_accuracy(y_true, predictions)
    return result
    
result = evaluate.measure_runtime(run)
print(f"Accuracy: {result:.4f}")